Example Google Colab script used to reproduce one of the benchmark cases from
  *Poulet, T. & Behnoudfar, P., "Physics-informed neural network reconciles Australian displacements and tectonic stresses", Scientific Reports (2023)*

Script deploys SciANN (https://www.sciann.com), based on SciANN-SolidMechanics.py from https://github.com/sciann/sciann-applications/tree/master/SciANN-SolidMechanics by Ehsan Haghighat.

Please note that this software is provided as-is, and there is no guarantee of future updates or support. The developers of this software are not responsible for any misuse or improper use of the software.

 Imports, using specific versions of Python and tensorflow

In [ ]:
!pip install tensorflow==2.11

(Restart the runtime before continuing)

In [1]:
# This script was tested with Python 3.10 and tensorflow 2.11
import sys
assert sys.version[0:4] == '3.10', "Using wrong Python version {0}".format(sys.version)
import tensorflow as tf
assert tf.__version__[0:4] == '2.11', "Using wrong tensorflow version {0}".format(tf.__version__)

Install sciann (https://www.sciann.com)

In [ ]:
try:
  import sciann
  print('SciANN available')
except ImportError:
  print('Installing SciANN...')
  !pip install sciann
  import sciann
# This script was tested with SciANN 0.7.0.1
print('Using SciANN version {0}'.format(sciann.__version__))

In [3]:
import argparse, csv, os, time, sys
import numpy as np
from numpy import linalg as LA
from numpy.linalg import norm
from sciann.utils.math import diff
from sciann import SciModel, Functional, Parameter
from sciann import Data, Tie
from sciann import Variable, Field
from sciann import atan2, pow, sqrt
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt

In [ ]:
pi = np.pi

# Parameter values to compare with analytical solution
lmbd = 1.0
mu = 0.5
qload = 4.0

# Geometry
XMIN, XMAX = 0.0, 1.0
YMIN, YMAX = 0.0, 1.0

# SciANN parameters
parser = argparse.ArgumentParser()
parser.add_argument('-l', '--layers', help='Num layers and neurons (default 4 layers each 40 neurons [40, 40, 40, 40])', type=int, nargs='+', default=[40]*4)
parser.add_argument('-af', '--actf', help='Activation function (default tanh)', type=str, nargs=1, default=['tanh'])
parser.add_argument('-nx', '--numx', help='Num Node in X (default 20)', type=int, nargs=1, default=[20])
parser.add_argument('-ny', '--numy', help='Num Node in Y (default 20)', type=int, nargs=1, default=[20])
parser.add_argument('-bs', '--batchsize', help='Batch size for Adam optimizer (default 32)', type=int, nargs=1, default=[32])
parser.add_argument('-e', '--epochs', help='Maximum number of epochs (default 10000)', type=int, nargs=1, default=[10000])
parser.add_argument('-lr', '--learningrate', help='Initial learning rate (default 0.001)', type=float, nargs=1, default=[[0, 2000, 4000], [2e-3, 1e-3, 1e-4]])
parser.add_argument('-in', '--independent_networks', help='Use independent networks for each var (default False)', type=bool, nargs=1, default=[False])
parser.add_argument('-v', '--verbose', help='Show training progress (default 2) (check Keras.fit)', type=int, nargs=1, default=[2])

parser.add_argument('--shuffle', help='Shuffle data for training (default True)', type=bool, nargs=1, default=[True])
parser.add_argument('--stopafter', help='Patience argument from Keras (default 500)', type=int, nargs=1, default=[500])
parser.add_argument('--savefreq', help='Frequency to save weights (each n-epoch)', type=int, nargs=1, default=[100000])
parser.add_argument('--dtype', help='Data type for weights and biases (default float64)', type=str, nargs=1, default=['float64'])
parser.add_argument('--gpu', help='Use GPU if available (default False)', type=bool, nargs=1, default=[False])
parser.add_argument('-op', '--outputpath', help='Output path (default ./file_name)', type=str, nargs=1, default=['output-benchmark'])
parser.add_argument('-of', '--outputprefix', help='Output path (default res**)', type=str, nargs=1, default=['res'])

parser.add_argument('-nxp', '--numxplot', help='Num Node in X for ploting final results (default 200)', type=int, nargs=1, default=[200])
parser.add_argument('-nyp', '--numyplot', help='Num Node in Y for ploting final results (default 200)', type=int, nargs=1, default=[200])
parser.add_argument('--plot', help='Plot the model', nargs='?', default=False)

parser.add_argument('--case', help='Benchmark case/scenario index (default 1)', type=int, nargs=1, default=[1])

sys.argv=[''] # mimicking arguments in interactive session (Google Colab)
args = parser.parse_args()
if not args.gpu[0]:
  os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print('os.environ["CUDA_VISIBLE_DEVICES"]={}'.format(os.environ["CUDA_VISIBLE_DEVICES"]))

def bodyfx(xx):
  x, y = xx[0], xx[1]
  Q = qload
  frc = - lmbd*(4*pi**2*np.cos(2*pi*x)*np.sin(pi*y) - Q*y**3*pi*np.cos(pi*x)) \
        - mu*(pi**2*np.cos(2*pi*x)*np.sin(pi*y) - Q*y**3*pi*np.cos(pi*x)) \
        - 8*mu*pi**2*np.cos(2*pi*x)*np.sin(pi*y)
  return frc

def bodyfy(xx):
  x, y = xx[0], xx[1]
  Q = qload
  frc = lmbd*(3*Q*y**2*np.sin(pi*x) - 2*pi**2*np.cos(pi*y)*np.sin(2*pi*x)) \
        - mu*(2*pi**2*np.cos(pi*y)*np.sin(2*pi*x) + (Q*y**4*pi**2*np.sin(pi*x))/4) \
        + 6*Q*mu*y**2*np.sin(pi*x)
  return frc

def dispx(xx):
  x, y = xx[0], xx[1]
  return np.cos(2*pi*x) * np.sin(pi*y)

def dispy(xx):
  x, y = xx[0], xx[1]
  Q = qload
  return np.sin(pi*x) * Q * y**4/4

def strainxx(xx):
  x, y = xx[0], xx[1]
  Q = qload
  return -2*pi*np.sin(2*pi*x)*np.sin(pi*y)

def strainyy(xx):
  x, y = xx[0], xx[1]
  Q = qload
  return np.sin(pi*x)*Q*y**3

def strainxy(xx):
  x, y = xx[0], xx[1]
  Q = qload
  return 0.5*(pi*np.cos(2*pi*x)*np.cos(pi*y) + pi*np.cos(pi*x)*Q*y**4/4)

def stressxx(xx):
  return (lmbd+2*mu)*strainxx(xx) + lmbd*strainyy(xx)

def stressyy(xx):
  return (lmbd+2*mu)*strainyy(xx) + lmbd*strainxx(xx)

def stressxy(xx):
  return 2.0*mu*strainxy(xx)

def myStressOrientation(xx):
  ''' Observations regarding stress orientation (in radian)
      using analytical solution
  '''
  x, y = xx[0], xx[1]
  sxx = stressxx(xx).flatten()
  sxy = stressxy(xx).flatten()
  syy = stressyy(xx).flatten()
  tmp = np.stack([sxx, sxy, sxy, syy], axis=-1)
  n = sxx.shape[0]
  tmp = tmp.reshape(n, 2, 2)
  eigenvalues, eigenvectors = LA.eig(tmp)
  i_mins = np.argmin(eigenvalues, axis=-1)
  eigv1 = eigenvectors[np.arange(n), :, i_mins]
  angles = np.arctan2(eigv1[:,1], eigv1[:,0])
  az = getShortAngleDifference(angles, np.zeros_like(angles))
  az2 = az.reshape(-1, 1)
  return az2

def cust_pcolor(AX, X, Y, C, title):
  im = AX.pcolor(X, Y, C, cmap="jet")
  AX.axis("equal")
  AX.axis("off")
  if title:
    AX.set_title(title, fontsize=5, y=1., pad=2)
  colorbar = plt.colorbar(im, ax=AX, shrink=0.8)
  colorbar.ax.tick_params(labelsize=3)
  for tick in colorbar.ax.get_yticklabels(which='both'):
    tick.set_fontsize(5)
  for tick in colorbar.ax.get_xticklabels(which='both'):
    tick.set_fontsize(5)

def cust_quiver(AX, X, Y, U, V, title):
  AX.axis("equal")
  AX.axis("off")
  im = AX.quiver(X, Y, U, V, angles='xy', scale_units='xy')
  AX.set_title(title, fontsize=5, y=1., pad=2)

def cust_plot(AX, X, Y, title, xlabel, ylabel):
  im = AX.plot(X, Y)
  AX.set_title(title, fontsize=5)
  if xlabel is not None: AX.set_xlabel(xlabel, fontsize=5)
  if ylabel is not None: AX.set_ylabel(ylabel, fontsize=5)
  for tick in AX.get_yticklabels(which='both'):
    tick.set_fontsize(5)
  for tick in AX.get_xticklabels(which='both'):
    tick.set_fontsize(5)

def cust_semilogx(AX, X, Y, xlabel, ylabel):
  im = []
  if X is None:
    im = AX.semilogy(Y, c='k', lw=1)
  else:
    im = AX.semilogy(X, Y, c='k', lw=1)
  if xlabel is not None: AX.set_xlabel(xlabel, fontsize=5)
  if ylabel is not None: AX.set_ylabel(ylabel, fontsize=5)
  for tick in AX.get_yticklabels(which='both'):
    tick.set_fontsize(5)
  for tick in AX.get_xticklabels(which='both'):
    tick.set_fontsize(5)

def getShortAngleDifference(alpha, beta):
  ''' Compute short angle difference between 2 directions
      @param alpha[in] - np.array (float), angles in radian
      @param beta[in] - np.array (float), angles in radian
  '''
  return (((alpha-beta)%np.pi)+0.5*np.pi)%np.pi - 0.5*np.pi

def mseOnShortAngle(y_true, y_pred):
  ''' Loss function with Mean Squared Error when inputs are angles and we want
    to compare the shortest angle differences modulo pi.
    y_true and y_pred are tensors.
  '''
  short_angle = tf.math.floormod(tf.math.floormod(y_true - y_pred, pi) + pi/2., pi) - pi/2.
  return K.mean(K.square(short_angle), axis=-1)

def getFileName(args):
  ''' Get root file name for outputs and save files of training results '''
  out_rootfilename = os.path.join(args.outputpath[0],
    args.outputprefix[0] + "_case{}".format(args.case[0]))
  return out_rootfilename

def train():
  ''' Train for specific benchmark case/scenario '''
  supported_cases = [1, 2, 3, 4, 5, 6, 7]
  assert args.case[0] in supported_cases, 'invalid case "{0}". Supported cases are {1}'\
    .format(args.case[0], supported_cases)

  # define output folder.
  if not os.path.isdir(args.outputpath[0]):
    os.mkdir(args.outputpath[0])
  fname = getFileName(args)

  # Neural Network Setup.
  x = Variable("x", dtype=args.dtype[0])
  y = Variable("y", dtype=args.dtype[0])

  if args.independent_networks[0]:
    Uxy = Functional("Uxy", [x, y], args.layers, args.actf[0])
    Vxy = Functional("Vxy", [x, y], args.layers, args.actf[0])
    Sxx = Functional("Sxx", [x, y], args.layers, args.actf[0])
    Syy = Functional("Syy", [x, y], args.layers, args.actf[0])
    Sxy = Functional("Sxy", [x, y], args.layers, args.actf[0])
  else:
    Uxy, Vxy, Sxx, Syy, Sxy = Functional(
      ["Uxy", "Vxy", "Sxx", "Syy", "Sxy"],
      [x, y], args.layers, args.actf[0])

  lame1 = lmbd
  lame2 = mu

  C11 = (2*lame2 + lame1)
  C12 = lame1
  C33 = 2*lame2

  Exx = diff(Uxy, x)
  Eyy = diff(Vxy, y)
  Exy = (diff(Uxy, y) + diff(Vxy, x))*0.5

  # Define constraints
  d1 = Data(Uxy)
  d2 = Data(Vxy)
  d3, d4 = None, None
  if args.case[0] in [1, 2]:
    d3 = Data(Sxx)
    d4 = Data(Syy)

  eigenval, az = None, None
  if args.case[0] in [2, 3, 4, 5, 6]:
    eigenval = 0.5 * ( Sxx + Syy - sqrt( pow(Sxx - Syy, 2) + 4 * pow(Sxy, 2) ) )
    az = Data( atan2(Sxy, eigenval - Syy) ) # stress orientation angle (azimuth)

  c1 = Tie(Sxx, Exx*C11 + Eyy*C12)
  c2 = Tie(Syy, Eyy*C11 + Exx*C12)
  c3 = Tie(Sxy, Exy*C33)

  Lx = diff(Sxx, x) + diff(Sxy, y)
  Ly = diff(Sxy, x) + diff(Syy, y)

  # Define the optimization model (set of inputs and constraints) per case
  targets = [d1, d2]
  loss_funcs = ["mse", "mse"]
  if args.case[0] in [1, 2]:
    targets += [d3, d4] # add stress values targets
    loss_funcs += ["mse", "mse"]
  if args.case[0] in [2, 3, 4, 5, 6]:
    targets += [az] # add stress orientations targets
    loss_funcs += [mseOnShortAngle]
  targets += [c1, c2, c3, Lx, Ly]
  loss_funcs += ["mse", "mse", "mse", "mse", "mse"]
  model = SciModel(
    inputs=[x, y],
    targets=targets,
    loss_func=loss_funcs
  )
  with open("{}_summary".format(fname), "w") as fobj:
    model.summary(print_fn=lambda x: fobj.write(x + '\n'))

  # Prepare training data
  ## Training grid
  Xmesh = np.linspace(XMIN, XMAX, args.numx[0]).reshape((-1, 1))
  Ymesh = np.linspace(YMIN, YMAX, args.numy[0]).reshape((-1, 1))
  X, Y = np.meshgrid(Xmesh, Ymesh)

  input_data = [X.reshape(-1, 1), Y.reshape(-1, 1)]

  # Get various boundary sets
  XTOL, YTOL = np.array([XMAX-XMIN, YMAX-YMIN])*1e-6
  left_ids = np.where(abs(input_data[0] - XMIN) < XTOL)[0]
  right_ids = np.where(abs(input_data[0] - XMAX) < XTOL)[0]
  bot_ids = np.where(abs(input_data[1] - YMIN) < YTOL)[0]
  top_ids = np.where(abs(input_data[1] - YMAX) < YTOL)[0]
  BC_ids_all = np.unique(np.concatenate([left_ids, right_ids, bot_ids, top_ids]))
  BC_ids_sid = np.unique(np.concatenate([left_ids, right_ids]))
  BC_ids_bot = np.unique(np.concatenate([bot_ids]))
  BC_ids_top = np.unique(np.concatenate([top_ids]))
  BC_ids_top_bot = np.unique(np.concatenate([top_ids, bot_ids]))
  BC_ids_bot_sides = np.unique(np.concatenate([bot_ids, left_ids, right_ids]))
  BC_ids_botpt1 = np.where(abs(input_data[0] - XMIN) + abs(input_data[1] - YMIN)  < XTOL)[0]
  BC_ids_botpt2 = np.where(abs(input_data[0] - XMAX) + abs(input_data[1] - YMIN)  < XTOL)[0]
  BC_ids_toppt2 = np.where(abs(input_data[0] - XMAX) + abs(input_data[1] - YMAX)  < XTOL)[0]
  BC_ids_botcorners = np.unique(np.concatenate([BC_ids_botpt1, BC_ids_botpt2]))
  BC_ids_oppcorners = np.unique(np.concatenate([BC_ids_botpt1, BC_ids_toppt2]))

  ## data associated to constrains defined earlier
  # Define constraints
  data_d1 = dispx(input_data) # Dirichlet BC in u_x
  data_d2 = dispy(input_data) # Dirichlet BC in u_y
  data_d3, data_d4 = None, None
  if args.case[0] in [1, 2]:
    data_d3 = stressxx(input_data)
    data_d4 = stressyy(input_data)
  data_az = None
  if args.case[0] in [2, 3, 4, 5, 6]:
    data_az = myStressOrientation(input_data) # stress orientation data
  data_c1 = 'zeros'
  data_c2 = 'zeros'
  data_c3 = 'zeros'
  data_Lx = bodyfx(input_data)
  data_Ly = bodyfy(input_data)

  BC_ids_disp_x, BC_ids_disp_y = None, None
  if args.case[0] in [1, 2, 3]:
    BC_ids_disp_x = BC_ids_top_bot
    BC_ids_disp_y = BC_ids_bot_sides
  elif args.case[0] in [4]:
    BC_ids_disp_x = BC_ids_bot
    BC_ids_disp_y = BC_ids_bot
  elif args.case[0] in [5]:
    BC_ids_disp_x = BC_ids_botcorners
    BC_ids_disp_y = BC_ids_botcorners
  elif args.case[0] in [6, 7]:
    BC_ids_disp_x = BC_ids_oppcorners
    BC_ids_disp_y = BC_ids_oppcorners

  target_data = [(BC_ids_disp_x, data_d1), # disp_x at some boundaries
                 (BC_ids_disp_y, data_d2)] # disp_y at some boundaries
  if args.case[0] in [1, 2]:
    target_data += [(BC_ids_sid, data_d3), # Sxx at some boundaries
                    (BC_ids_top, data_d4)] # Syy at some boundaries
  if args.case[0] in [2, 3, 4, 5, 6]:
    target_data += [data_az] # stress orientation at all test points
  target_data += [data_c1, data_c2, data_c3, # constitutive model at all test points
                  data_Lx, data_Ly] # body force at all test points

  # Train the model
  training_time = time.time()
  history = model.train(
    x_true=input_data,
    y_true=target_data,
    epochs=args.epochs[0],
    batch_size=args.batchsize[0],
    shuffle=args.shuffle[0],
    learning_rate=args.learningrate,
    stop_after=args.stopafter[0],
    verbose=args.verbose[0],
  )
  training_time = time.time() - training_time

  for loss in history.history:
    np.savetxt(fname+"_{}".format("_".join(loss.split("/"))),
                np.array(history.history[loss]).reshape(-1, 1))

  time_steps = np.linspace(0, training_time, len(history.history["loss"]))
  np.savetxt(fname+"_Time", time_steps.reshape(-1,1))

  # Post process the trained model.
  Xmesh_plot = np.linspace(XMIN, XMAX, args.numxplot[0]).reshape((-1, 1))
  Ymesh_plot = np.linspace(YMIN, YMAX, args.numyplot[0]).reshape((-1, 1))
  X_plot, Y_plot = np.meshgrid(Xmesh_plot, Ymesh_plot)
  input_plot = [X_plot.reshape(-1, 1), Y_plot.reshape(-1, 1)]

  Uxy_pred = Uxy.eval(model, input_plot)
  Vxy_pred = Vxy.eval(model, input_plot)
  Exx_pred = Exx.eval(model, input_plot)
  Eyy_pred = Eyy.eval(model, input_plot)
  Exy_pred = Exy.eval(model, input_plot)
  Sxx_pred = Sxx.eval(model, input_plot)
  Syy_pred = Syy.eval(model, input_plot)
  Sxy_pred = Sxy.eval(model, input_plot)

  np.savetxt(fname+"_Xmesh", X_plot, delimiter=', ')
  np.savetxt(fname+"_Ymesh", Y_plot, delimiter=', ')
  np.savetxt(fname+"_Uxy", Uxy_pred.reshape(X_plot.shape), delimiter=', ')
  np.savetxt(fname+"_Vxy", Vxy_pred.reshape(X_plot.shape), delimiter=', ')
  np.savetxt(fname+"_Exx", Exx_pred.reshape(X_plot.shape), delimiter=', ')
  np.savetxt(fname+"_Eyy", Eyy_pred.reshape(X_plot.shape), delimiter=', ')
  np.savetxt(fname+"_Exy", Exy_pred.reshape(X_plot.shape), delimiter=', ')
  np.savetxt(fname+"_Sxx", Sxx_pred.reshape(X_plot.shape), delimiter=', ')
  np.savetxt(fname+"_Syy", Syy_pred.reshape(X_plot.shape), delimiter=', ')
  np.savetxt(fname+"_Sxy", Sxy_pred.reshape(X_plot.shape), delimiter=', ')

def plot(do_show=True):
  fname = getFileName(args)

  ### Plot convergence
  loss = np.loadtxt(fname+"_loss")
  time = np.loadtxt(fname+"_Time")
  lr = np.loadtxt(fname+"_lr")
  fig, ax = plt.subplots(1, 2, figsize=(4, 2), dpi=300)
  cust_semilogx(ax[0], None, loss/loss[0], "epochs", "L/L0")
  cust_semilogx(ax[1], None, lr, "epochs", "learning rate")
  fig.subplots_adjust(left=0.1, right=0.97, bottom=0.15, top=0.9, wspace=0.6, hspace=0.2)
  plt.savefig("{}_loss.png".format(fname))

  Xmesh = np.loadtxt(fname+"_Xmesh", delimiter=',')
  Ymesh = np.loadtxt(fname+"_Ymesh", delimiter=',')

  ### Plot results
  fig, ax = plt.subplots(3, 2, figsize=(2., 3.), dpi=300)
  Ux = np.loadtxt(fname+"_Uxy", delimiter=',')
  Uy = np.loadtxt(fname+"_Vxy", delimiter=',')
  Utotal = np.sqrt(Ux*Ux + Uy*Uy)
  Ux_ana = dispx([Xmesh, Ymesh]) # analytical solution
  Uy_ana = dispy([Xmesh, Ymesh]) # analytical solution
  Ux_err = Ux - Ux_ana
  Uy_err = Uy - Uy_ana
  L2_relerr_ux = norm(Ux_err, ord=2) / norm(Ux_ana, ord=2)
  L2_relerr_uy = norm(Uy_err, ord=2) / norm(Uy_ana, ord=2)
  Linf_err_ux = np.abs(Ux_err).max()
  Linf_err_uy = np.abs(Uy_err).max()
  print("The minimum relative loss is", min(loss)/loss[0])
  print("The L_inf error on u_x is", Linf_err_ux)
  print("The L_inf error on u_y is", Linf_err_uy)
  print("The L_2 relative error on u_x is", L2_relerr_ux)
  print("The L_2 relative error on u_y is", L2_relerr_uy)
  cust_pcolor(ax[0,0], Xmesh, Ymesh, Ux, "Ux")
  cust_pcolor(ax[0,1], Xmesh, Ymesh, Uy, "Uy")
  cust_pcolor(ax[1,0], Xmesh, Ymesh, Ux_err, "Ux error")
  cust_pcolor(ax[1,1], Xmesh, Ymesh, Uy_err, "Uy error")
  cust_pcolor(ax[2,0], Xmesh, Ymesh, myStressOrientation([Xmesh, Ymesh]).reshape(Xmesh.shape), "data azimuth")
  Sxx_pred = np.loadtxt(fname+"_Sxx", delimiter=',')
  Sxy_pred = np.loadtxt(fname+"_Sxy", delimiter=',')
  Syy_pred = np.loadtxt(fname+"_Syy", delimiter=',')
  eigenval_pred = 0.5 * ( Sxx_pred  + Syy_pred  - np.sqrt( np.square(Sxx_pred - Syy_pred) + 4 * np.square(Sxy_pred) ) )
  az_pred = np.arctan2(Sxy_pred , eigenval_pred - Syy_pred) # stress orientation angle (azimuth)
  az_pred2 = getShortAngleDifference(az_pred, np.zeros_like(az_pred))
  cust_pcolor(ax[2,1], Xmesh, Ymesh, az_pred2, "Predicted azimuth")
  plt.subplots_adjust(left=0.01, right=0.9, bottom=0.01, top=0.92, hspace=0.4)
  plt.savefig("{}_disp.png".format(fname))

  if do_show:
    plt.show()

class Timer(object):
  def __init__(self, name=None):
    self.name = name

  def __enter__(self):
    self.tstart = time.time()

  def __exit__(self, type, value, traceback):
    if self.name:
      print('[%s]' % self.name,)
    print('Elapsed: %.2f s' % (time.time() - self.tstart))

if args.plot==False:
  with Timer('Training'):
    train()
  from google.colab import output
  output.clear()
  plot(do_show=False)
else:
  plot(do_show=True)